In [ ]:
from datasets import SOCOFingGender, SOCOFingFingers, SOCOFingSubjects
from FPMLmodule.backbones import ResNet50
from FPMLmodule.classifiers import ResNetClassifier
from FPMLmodule.fpml import FPML 
from FPMLmodule.utils import displayDsSamples, displayDsSplit, displayTrainProgress
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
# from keras import layers, Sequential, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

In [ ]:
# Global Config
seed=9
img_dim = (120, 120, 3)
img_height, img_width, img_channels = img_dim
batch_size = 32

# Dataset configuration

AUTOTUNE = tf.data.AUTOTUNE
split_ratio = [0.7, 0.15, 0.15]
shuffle=True
dsConfig = {
    'batchSize': batch_size, 
    'parallelTune': AUTOTUNE, 
    'split': split_ratio, 
    'inputDim': img_dim, 
    'seed': seed, 
    'shuffle': shuffle
    }
weights = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Model Config
learning_rate = 0.001
epochs_find_best = 10
epochs_best = 100



print(img_dim)
print(img_height, img_width, img_channels)

genderClassNames=['F','M']
fingersClassNames=['R_thumb', 'R_index', 'R_middle', 'R_ring', 'R_little', 'L_thumb', 'L_index', 'L_middle', 'L_ring', 'L_little ']

In [ ]:
SOCOGender = SOCOFingGender(**dict(dsConfig, sampling=SOCOFingGender.UNDER_SAMPLING))
SOCOSubjects = SOCOFingSubjects(**dsConfig)
SOCOFingers = SOCOFingFingers(**dsConfig)

Dataset split check

In [ ]:
genderDS = SOCOGender.createDatasets()
subjectDS = SOCOSubjects.createDatasets()
fingersDS = SOCOFingers.createDatasets()

In [ ]:
unionSet = set()
interSet = set()
sets = []
for ds in subjectDS:
    dsSet = set()
    for _, labels in ds.as_numpy_iterator():
        for label in labels:
            parseLabel = label.decode('utf-8')
            dsSet.add(parseLabel)
    sets.append(dsSet)
    unionSet = unionSet.union(dsSet) 
    interSet = interSet.intersection(dsSet) 
    
print('Union of splitted ds subject: {}, intersaction: {}'.format(len(unionSet), len(interSet)))

In [ ]:
dsNames = ['SOCOGender  ', 'SOCOSubjects', 'SOCOFingers ']
splitNames = ['train', 'test ', 'valid']
datasets = [genderDS, subjectDS, fingersDS]

data = []
for ds in datasets:
    data.append([len(i) for i in ds])

data = []
for ds in datasets:
    splitLens = []
    for split in ds:
        splitLen = 0
        for _, labels in split.as_numpy_iterator():
            splitLen+=len(labels)
        splitLens.append(splitLen)
    data.append(splitLens)

row_format ="{:>15}" * (len(dsNames) + 2)
print(row_format.format("", *splitNames, 'total'))
for name, row in zip(dsNames, data):
    print(row_format.format(name, *row, np.sum(row)))

In [ ]:
displayDsSplit(genderDS)

In [ ]:
displayDsSamples(genderDS[0], 'Gender - Train', (5, 5), classNames=genderClassNames)

In [ ]:
train_ds, test_ds, val_ds = genderDS

rn50 = ResNet50(img_dim, weights=weights, trainable=False)
rnc = ResNetClassifier(2, "softmax")

fpml = FPML(rn50, rnc, "", img_dim)

model = fpml.createModel(Adam, learning_rate, 'binary_crossentropy', 'accuracy')

model_history = model.fit(train_ds, validation_data=val_ds, epochs=10)

In [ ]:
test_history = model.evaluate(test_ds)

In [ ]:
predicted = model.predict(test_ds)

y_test = np.concatenate([y for x, y in test_ds], axis=0)
    
y_test = np.argmax(y_test, axis=1).astype(np.uint8)
y_predict = np.argmax(predicted, axis=1).astype(np.uint8)

cm = confusion_matrix(y_test, y_predict)
sn.heatmap(cm ,annot = True, fmt='g')
acc = (np.array(y_test) == np.array(y_predict)).sum()/len(y_test)
plt.title("Test Set Accuracy: {}%".format(str(round(acc*100, 2))))
plt.show()

In [ ]:
# def displayConfusion(dataset, model)
test_images = []
y_test = []
y_predict = []

for images, labels in test_ds:

    tmp_predict = model.predict(images)
    tmp_true_max = np.argmax(labels.numpy(), axis=1).astype(np.uint8)
    tmp_pred_max = np.argmax(tmp_predict, axis=1).astype(np.uint8)
    test_images = [*test_images, *images]
    y_test = [*y_test, *tmp_true_max]
    y_predict = [*y_predict, *tmp_pred_max]


cm = confusion_matrix(y_test, y_predict)
sn.heatmap(cm ,annot = True, fmt='g')
acc = (np.array(y_test) == np.array(y_predict)).sum()/len(y_test)
plt.title("Test Set Accuracy:  "+str(round(acc*100, 2)))
plt.show()
    

In [ ]:
displayTrainProgress(model_history)